# Movie/TV Shows Sharing Actors with Your Favorite Movie/Show

In this blog post, I’m going to make a super cool web scraper. 

Here’s a link to my project repository. https://github.com/Tiana0117/imdb_recommendation 

Here’s how we set up the project. We will go through it step by step.

## Set Up
First, we need to locate the IMDB pages for future implementation.

* **Step 1**: Pick your favorite movie or TV show
* **Step 2**: Locate its IMDB page. 
In my case, I picked my favorite TV show as *Avengers: Endgame*. Its IMDB page is at: https://www.imdb.com/title/tt4154796/
* **Step 3**: Click on the Cast & Crew link. This will take you to a page with URL of the form: `<original_url>fullcredits/`
* **Step 4**: Scroll until you see the Series Cast section. Click on the portrait of one of the actors. This will take you to a page with a different-looking URL. My favorite actor is Robert Downey Jr. , and his URL is https://www.imdb.com/name/nm0000375/
* **Step 5**: Once reaching the actor page, scroll down until you see the actor’s Filmography section. Note the titles of a few movies and TV shows in this section.

Above, we have locate several IMDB pages. And in the following section, we are going to use web scraper to mimic this process.
> Start with your favorite movie or TV show

> Look at all the actors in that movie or TV show 

> Log all the other movies or TV shows that they worked on

## Initialize Project
Open a terminal in the location of your repository on your laptop, and type:

In [ ]:
conda activate PIC16B
scrapy startproject IMDB_scraper
cd IMDB_scraper

Then, add the following line to the file `settings.py`:

In [ ]:
CLOSESPIDER_PAGECOUNT = 20

This line just prevents your scraper from downloading too much data while you’re still testing things out. You’ll remove this line later.

Create a file inside the `spiders` directory called `imdb_spider.py`. Add the following lines to the file:

In [ ]:
# to run 
# scrapy crawl imdb_spider -o movies.csv

import scrapy
from scrapy.spiders import Spider
from scrapy.http import Request
from scrapy.linkextractors import LinkExtractor
from re import search

class ImdbSpider(scrapy.Spider):
    name = 'imdb_spider'
    
    start_urls = ['https://www.imdb.com/title/tt4154796/']

Replace the entry of `start_urls` with the URL corresponding to your favorite movie or TV show.

Now, implement three parsing methods for the `ImdbSpider` class.

**Navigate to the Cast & Crew Page:**

The first parse method is `parse(self,response)`. 
* This method assumes that you start on a movie page, and then navigate to the Cast & Crew page. Cast & Crew page has url <movie_url>fullcredits. 
* Once there, the `parse_full_credits(self,response)` should be called, by specifying this method in the `callback` argument to a yielded `scrapy.Request`. 
* The `parse()` method does not return any data.

In [ ]:
def parse(self,response):
    """
    This method is to navigate from the starting movie page to the Cast & Crew Page.
    The Cast & Crew Page has the url of <movie_url>fullcredits.
    Once there, the method parse_full_credits(self,response) is called.

    """
    # get part of the url address specifying the specific actor
    actor_suffix = [a.attrib["href"] for a in response.css("td.primary_photo a")]
    prefix = "https://www.imdb.com/"
        
    # the full url is prefix + actor-specified id
    actor_url = [prefix + suffix for suffix in actor_suffix]
        
    # yield the actor_url, and call the parse_actor_page(self,response)
    for actor in actor_url:

        yield Request(actor, callback = self.parse_actor_page)

**Navigate to the actor page:**

Then, we define the parsing method `parse_full_credits(self, response)`. 
* `parse_full_credits(self, response)` assumes that you start on the Cast & Crew page. 
* Its purpose is to yield a `scrapy.Request` for the page of each actor listed on the page. Crew members are not included. 
* Once the actor's page is reached, the yielded request specify the method `parse_actor_page(self, response)` being called. 
* The `parse_full_credits()` method does not return any data. 

In [ ]:
def parse_full_credits(self, response):
   """
    This method is to yield a dictionary with actor_names and movie_or_TV names as values.
    The dictionary has each of the movies or TV shows on which that actor has worked.
    """

    # get the actor name
    actor_name = response.css("span.itemprop::text").get()
        
    # get the movie_or_TV_name
    for movie in response.css("div.filmo-row"):
        movie_or_TV_name = [movie.css("a::text").get()]
        
        # yield the dictionary
        yield {
            "actor" : actor_name, 
            "movie_or_TV_name" : movie_or_TV_name
        }        

**Yield a dictionary with the actor name and the movie or TV show name:**

Then we come to the last parsing method. 
* `parse_actor_page(self, response)` assumes that you start on the page of an actor. 
* It yields a dictionary with two key-value pairs, of the form `{"actor" : actor_name, "movie_or_TV_name" : movie_or_TV_name}`. 
* The method should yield one such dictionary for each of the movies or TV shows on which that actor has worked. Note that you will need to determine both the name of the actor and the name of each movie or TV show. 

After the above three parsing methods being implemented, run the command in the terminal.

In [ ]:
scrapy crawl imdb_spider -o results.csv

## Make Recommendations

In this section, we want to make recommendations based on the number of shared actors. To present movies and the number of shared actors with our favortie movie/TV shows, I used `dataframe` and `scatterplot`.

Once you have successfully tested your `imdb_spider.py` file, you should have a csv file generated in the same directory of your file. Then, comment out the following line in the `settings.py` file.

In [ ]:
CLOSESPIDER_PAGECOUNT = 20

Then, run the command below. Before running this command, delete the csv file generated before in case of overwriting erros.

In [ ]:
scrapy crawl imdb_spider -o results.csv

Now you get a csv file called `results.csv`, with columns for actor names and the movies and TV shows on which they worked.

Then, we want to make recommendations based on this csv. We compute a sorted list with the top movies and TV shows that share actors with your favorite movie or TV show. 

In [1]:
# import necessary library
import pandas as pd

We first read the csv file into our jupyter notebook.

In [4]:
# read csv as dataframe
result = pd.read_csv("/Users/yiningliang/PIC16B Gitthub/imdb_recommendation/IMDB_scraper/python and csv/results.csv")

In [5]:
result.head()

,actor,movie_or_TV_name
0,Ty Simpkins,Growing up Gorman
1,Ty Simpkins,The Re-Education of Molly Singer
2,Ty Simpkins,The Whale
3,Ty Simpkins,Where's Rose
4,Ty Simpkins,Chimerica


We use the method `value_counts()`. This method returns a Series containing counts of unique values in the dataframe.

In [7]:
count = result.value_counts(subset = "movie_or_TV_name",
                     sort = True,
                     ascending = False)
count.head()

movie_or_TV_name
Avengers: Endgame         187
Entertainment Tonight      90
Avengers: Infinity War     74
Celebrity Page             70
Made in Hollywood          69
dtype: int64

Then, we format the Series into a dataframe, with column name specified.

In [8]:
# series to dataframe
df_recommend = count.to_frame(name = "number of shared actors")
df_recommend = df_recommend.reset_index()

Here’s the dataframe I obtained for *Avengers: Endgame*. Of course, most shows will “share” the most actors with themselves.

In [9]:
# df_recommend inspection
df_recommend

,movie_or_TV_name,number of shared actors
0,Avengers: Endgame,187
1,Entertainment Tonight,90
2,Avengers: Infinity War,74
3,Celebrity Page,70
4,Made in Hollywood,69
...,...,...
9330,James Bond's 'Spectre' with Jonathan Ross,1
9331,James Blake: My Willing Heart,1
9332,Jaguar: Rendezvous,1
9333,Jackman,1


Then, we create a plot to visualize top 10 movie or TV shows sharing the most same actors. We use `plotly` to help create an interactive plot.

In [13]:
from plotly import express as px
from matplotlib import pyplot as plt
from plotly.io import write_html

First, we want to extract the first 10 lines of the dataframe to prepare for the plot.

In [14]:
df_plot = df_recommend.head(10)

Then, we create the scatterplot with the dataframe `df_plot`. The `color` depends on the `number of shared actors`.

In [15]:
fig = px.scatter(df_plot,
                 x = "movie_or_TV_name",
                 y = "number of shared actors", 
                 color = "number of shared actors",
                 color_continuous_midpoint = 0,
                 title = "Scatterplot visualizing movies with shared actors")
    
write_html(fig, "movie_scatter.html")